# Test: Data Scientist (Client2) - SyftBox Mode

Automated test notebook for the Data Scientist workflow with SyftBox encryption.
Run with: `jupyter execute sc_test_ds.ipynb`

In [1]:
import beaver
beaver.dev() # install dev packages only in local dev mode
# beaver.show_debug()
# session.open() # open in finder

syftbox-sdk reinstalled from /Users/madhavajay/dev/biovault-desktop/workspace3/biovault/syftbox-sdk/python
beaver reinstalled from /Users/madhavajay/dev/biovault-desktop/workspace3/biovault/biovault-beaver/python

Restart kernel to use new versions.


In [2]:
!uv pip install scikit-misc scanpy -q
import beaver
import scanpy as sc # really slow first load

In [3]:
bv = beaver.ctx()
session = bv.active_session()

🔄 Auto-load replies enabled for client1@sandbox.local (polling every 2.0s)
🟢 Active session loaded: c4a9ae0bd0f6
   Peer: client2@sandbox.local


In [4]:
session

🟢 Session(
    id='c4a9ae0bd0f6',
    peer='client2@sandbox.local',
    role='peer',
    status='active'
)

In [5]:
session.peer_remote_vars

RemoteVarView(client2@sandbox.local): empty

In [6]:
session.wait_for_remote_var("number")

⏳ Waiting for 'number' from client2@sandbox.local...
📬 'number' is now available!
✓ Loaded Twin 'number' from published location


🌍 Twin: number (MOCK DATA - SAFE)
  🔒 Private    (not available) 💡 .request_private()
  🌍 Public    100    ← .value uses this
  Owner: client2@sandbox.local
  Live: ⚫ Disabled
  IDs: twin=a4a08761... private=50524aeb... public=246fb5a4...

In [7]:
session.peer_remote_vars["number"].load()

✓ Loaded Twin 'number' from published location


🌍 Twin: number (MOCK DATA - SAFE)
  🔒 Private    (not available) 💡 .request_private()
  🌍 Public    100    ← .value uses this
  Owner: client2@sandbox.local
  Live: ⚫ Disabled
  IDs: twin=a4a08761... private=50524aeb... public=246fb5a4...

In [8]:
number.private

In [9]:
number.public

100

In [10]:
@bv
def add_one(number):
    return number + 1

In [11]:
res = add_one(number)
res

⚠️  Twin: add_one_result (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━...    ← .value uses this
  🌍 Public    101    ✓
  Live: ⚫ Disabled
  IDs: twin=f44573a7... private=c991ddf9... public=100d3e33...

In [12]:
res.public

101

In [13]:
res.request_private()

📨 Sending computation request to client2@sandbox.local
   Function: add_one
   Result: res
📤 Writing to session folder: /Users/madhavajay/dev/biovault-desktop/workspace3/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/c4a9ae0bd0f6
✓ Sent to /Users/madhavajay/dev/biovault-desktop/workspace3/biovault/sandbox/client1@sandbox.local/datasites/client1@sandbox.local/shared/biovault/sessions/c4a9ae0bd0f6/aa4e5fce8a8d4a89ad9038d11b8df01a.beaver
💡 Result will auto-update when client2@sandbox.local approves


In [14]:
bv.wait_for_response(res)

⏳ Waiting for response to 'add_one_result'...
📬 Response received for 'add_one_result'
   From: client2@sandbox.local
✅ 'add_one_result' updated with result


⚠️  Twin: add_one_result (REAL + MOCK DATA)
  ⚠️  Be careful: This Twin contains both real and mock data
  🔒 Private    1338    ← .value uses this
  🌍 Public    101    ✓
  Live: ⚫ Disabled
  IDs: twin=f44573a7... private=c991ddf9... public=100d3e33...

In [15]:
res.private

1338

In [16]:
# session.open()

In [17]:
assert False # some small bugs below on waiting on object timing

AssertionError: 

In [ ]:
session.peer_remote_vars

In [ ]:
data = session.wait_for_remote_var("patient_sc", auto_accept=False)

In [ ]:
patient_sc = session.peer_remote_vars["patient_sc"].load(auto_accept=True)

In [ ]:
patient_sc

## Step 1: Violin Plot

In [ ]:
patient_sc.public

In [ ]:
# session.open() # open in finder

In [ ]:
@bv
def make_violin(adata):
    print("n_cells:", adata.n_obs)
    sc.pl.violin(
        adata, ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
        size=0,
        multi_panel=True
    )
    return plt.gcf()

violin_result = make_violin(patient_sc)

In [ ]:
violin_result

In [ ]:
# Display captured figures for mock run
violin_result.show_figures('public')

In [ ]:
print("[DS] Requesting private execution...")
violin_result.request_private()

In [ ]:
bv.wait_for_response(violin_result)

In [ ]:
TIMEOUT = 60
print("[DS] Waiting for violin result...")
env, _ = bv.wait_for_message(timeout=TIMEOUT, poll_interval=1.0)
assert env is not None, "Timeout waiting for violin result"

assert violin_result.private is not None, "Private result not received"
print(f"[DS] ✓ Violin private result received")
if violin_result.private_stdout:
    print(f"[DS] Private stdout: {violin_result.private_stdout[:50]}...")

In [ ]:
bv.inbox()[0].load()

In [ ]:
violin_result.show_figures("both")

## Step 2: Embedding Plot

In [ ]:
print("[DS] Running embedding analysis...")

@bv
def show_embedding(adata):
    print("n_cells:", adata.n_obs)
    return sc.pl.embedding(
        adata,
        basis='X_umap',
        color=['pct_counts_mt', 'cell_type'],
        ncols=1, size=15,
        frameon=False,
        show=False
    )

embedding_result = show_embedding(patient_sc)

assert embedding_result is not None, "Embedding computation returned None"
print(f"[DS] Embedding public figures: {len(embedding_result.public_figures)}")

In [ ]:
print("[DS] Requesting private execution...")
embedding_result.request_private()

print("[DS] Waiting for embedding result...")
env, _ = bv.wait_for_message(timeout=TIMEOUT, poll_interval=1.0)
assert env is not None, "Timeout waiting for embedding result"

print(f"[DS] ✓ Embedding private result received")

## Step 3: PCA Variance

In [ ]:
print("[DS] Running PCA variance analysis...")

@bv
def plot_pca_variance(adata):
    adata.layers['raw_counts'] = adata.X.copy()
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor='seurat_v3', layer='raw_counts', n_top_genes=5000)
    sc.pp.pca(adata, use_highly_variable=True)
    with plt.rc_context({'figure.figsize': (4, 3)}):
        sc.pl.pca_variance_ratio(adata, n_pcs=50)

pca_result = plot_pca_variance(patient_sc)

assert pca_result is not None, "PCA computation returned None"
print(f"[DS] PCA public figures: {len(pca_result.public_figures)}")

In [ ]:
print("[DS] Requesting private execution...")
pca_result.request_private()

print("[DS] Waiting for PCA result...")
env, _ = bv.wait_for_message(timeout=TIMEOUT, poll_interval=1.0)
assert env is not None, "Timeout waiting for PCA result"

print(f"[DS] ✓ PCA private result received")

## Step 4: UMAP Embedding

In [ ]:
print("[DS] Running UMAP embedding analysis...")

@bv
def umap_embedding(adata):
    sc.pp.neighbors(adata, n_pcs=50)
    sc.tl.umap(adata)
    return sc.pl.embedding(
        adata,
        basis='X_umap',
        color=['cell_type'],
        ncols=1, size=15,
        frameon=False,
        show=True
    )

umap_result = umap_embedding(patient_sc)

assert umap_result is not None, "UMAP computation returned None"
print(f"[DS] UMAP public figures: {len(umap_result.public_figures)}")

In [ ]:
print("[DS] Requesting private execution...")
umap_result.request_private()

print("[DS] Waiting for UMAP result...")
env, _ = bv.wait_for_message(timeout=TIMEOUT, poll_interval=1.0)
assert env is not None, "Timeout waiting for UMAP result"

print(f"[DS] ✓ UMAP private result received")

In [ ]:
# Final verification
print("\n" + "="*50)
print("[DS] TEST PASSED")
print("[DS] Successfully completed all 4 analysis steps:")
print("  1. Violin plot")
print("  2. Embedding plot")
print("  3. PCA variance")
print("  4. UMAP embedding")
print("="*50)

# Verify inbox has all results
inbox = bv.inbox()
print(f"\n[DS] Inbox contains {len(inbox)} messages")
assert len(inbox) >= 4, f"Expected at least 4 results in inbox, got {len(inbox)}"